<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Contents](Index.ipynb) | [Further Resources](03.13-Further-Resources.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.12-Performance-Eval-and-Query.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 高效率Pandas:eval()及query()

## 使用``pandas.eval()``進行高效率的操作

Pandas``eval()``函式使用字串敘述式高效率的計算使用了``DataFrame``的操作

In [7]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

In [ ]:
# 用Pandas計算總和
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 87.1 ms per loop


In [ ]:
# 用pd.eval()計算總和
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 42.2 ms per loop


### 支援``pd.eval()``的操作

In [8]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### 算術運算子
``pd.eval()``支援所有的算術運算子:

In [9]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2) # 相等回傳True

True

#### 比較運算子
``pd.eval()``支援所有的比較運算子，包括串接在一起的敘述:

In [10]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### 位元運算子
``pd.eval()``支援``&``以及``|``位元運算子:

In [11]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [12]:
# 也支援and以及or布林運算式:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### 物件屬性和索引

``pd.eval()``支援透過``obj.attr``語法存取物件屬性，以及使用``obj[index]``語法進行索引:

In [13]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

## 使用``DataFrame.eval()``進行逐欄操作

正如Pandas擁有一個高階的``pd.eval()``函式，``DataFrame``也有一個``eval()``方法可以使用同樣的方式運作。
``eval()``方法的好處是資料欄可以*用它的名字*來參考。

In [14]:
# 使用具有標籤的陣列
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [15]:
# 使用pd.eval()計算此3欄的敘述式
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

``DataFrame.eval()``方法對這些欄的計算更加簡潔:

In [16]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

在上述的程式碼中把*欄名當作是變數*看待，而結果也正如我們所料。

### 在DataFrame.eval()中的賦值運算

``DataFrame.eval()``也允許對任一個欄執行設定值的操作。

In [17]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


可以使用``df.eval()``去建立一個新的欄``'D'``，然後把來自於其他欄計算之後的值設定給它:

In [18]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


同樣的，任何已經存在的欄也可以被修改:

In [19]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### DataFrame.eval()的本地端變數

``DataFrame.eval()``也支援一個額外的語法使之可以操作本地端的Python變數

In [20]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

在``@``字元標記了它是一個*變數名稱*而不是*欄名*，可以有效率計算包含2個"名稱空間"的敘述式: 欄的名稱空間，以及Python物件的名稱空間。
要注意``@``字元只被``DataFrame.eval()``*方法*支援，``pandas.eval()`` *函式*則不行，因為``pandas.eval()``函式只能存取一個(Python)名稱空間。

## DataFrame.query() 方法

``DataFrame``有另外可使用字串形式計算，為``query()``方法

In [21]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

以之前在``DataFrame.eval()``中使用過的情形為例，這是一個包含有``DataFrame``欄位的敘述式，它不能使用``DataFrame.eval()``語法表示，然而，這一類型的過濾操作`可以使用``query()``方法:

In [22]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

此外，為了要更有效率運算，相較於使用遮罩敘述，此種方式可更容易被閱讀和理解。留意``query()``方法也可以接受``@``去標註本地端變數:

In [23]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## 效能:何時使用這些函式

當考量到何時要使用這些函式時，有2個主要考量點:
*運算時間*和*記憶體使用*

記憶體的使用是最需要預測的考量點，每一個複合的敘述式包含Numpy陣列或是Pandas的 ``DataFrame``將會造成隱含的暫時性陣列之建立。

In [24]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

In [25]:
# 大約等於以下的敘述
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

如果暫時的``DataFrame``對於可用記憶體(通常大都是幾個GB)比較起來是非常顯著的，就可使用``eval()``或``query()``。

In [26]:
# 確認陣列的大小
df.values.nbytes

32000

而在效能這一端，當還沒用完記憶體時，``eval()``會是比較快的。``eval()``可以避免掉一些潛在較慢的，在不同快取記憶體之間值的搬移。實務上，傳統方法和使用``eval``/``query``方法在運算時間上的差異並不明顯，如果有差的話，在小一點的陣列上，傳統的方法要來得快一些。``eval``/``query``的主要優勢在於節省記憶體，以及有時候它們提供比較清楚的語法。
